<a href="https://colab.research.google.com/github/mishabar410/ML-2022/blob/main/time_stretch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import librosa
import numpy as np

# Load the audio file
filename = 'test_mono.wav'
y, sr = librosa.load(filename, sr=None, mono=True)

# Set the stretching factor
stretch_factor = 1.5

# Set the window size and hop size
win_size = 2048
hop_size = int(win_size/4)

# Calculate the FFT of the signal
y_fft = librosa.stft(y, n_fft=win_size, hop_length=hop_size)

# Divide the signal into frames and calculate the phase angle
y_mag, y_phase = librosa.magphase(y_fft)
num_frames = y_mag.shape[1]

# Initialize the output signal
output = np.zeros_like(y_mag)

# Set the time indices for the output signal
time_indices = np.arange(0, num_frames, stretch_factor)

# Loop through the output time indices
for i, t in enumerate(time_indices):

    # Find the nearest integer frame index
    frame_index = int(round(t))

    # Interpolate the magnitude and phase of the signal
    if frame_index >= num_frames:
        # If the frame index is out of bounds, use the last frame
        mag = y_mag[:, -1]
        phase = y_phase[:, -1]
    else:
        # Otherwise, interpolate between frames
        mag = y_mag[:, frame_index-1] + (t-frame_index)*(y_mag[:, frame_index]-y_mag[:, frame_index-1])
        phase = y_phase[:, frame_index-1] + (t-frame_index)*(y_phase[:, frame_index]-y_phase[:, frame_index-1])

    # Reconstruct the complex spectrum of the signal
    y_stretch = mag * np.exp(1j*phase)

    # Add the stretched frame to the output signal
    output[:, i] = y_stretch

# Invert the FFT to get the time-domain signal
y_stretch = librosa.istft(output, hop_length=hop_size)

# Save the output signal to a new audio file

# import soundfile as sf
# import soundfile as sf
# sf.write('test_stretch.wav', 48000, 'PCM_24')

scipy.io.wavfile.write('test_stretch.wav', sr, y_stretch)

<ipython-input-56-1dbfcc969496>:48: ComplexWarning: Casting complex values to real discards the imaginary part
  output[:, i] = y_stretch


IndexError: ignored